In [ ]:
import pandas as pd
import geopandas
from os import path, environ, makedirs
import requests as re
from io import BytesIO
from dotenv import load_dotenv
from unidecode import unidecode
import numpy as np

In [ ]:
load_dotenv()

In [ ]:
def criar_mapper_subprefeituras(df: pd.DataFrame, subs_qlik: list, sub_col_name='região') -> dict:

    subs = df[sub_col_name].apply(unidecode).unique().tolist()
    subs.sort()
    return {s: q for s, q in zip(subs, subs_qlik)}

In [ ]:
def aplicar_mapper(df: pd.DataFrame, mapper: dict, sub_col_name='região') -> pd.DataFrame:
    
    df.insert(
        1,
        'sub.NOME',
        df[sub_col_name].apply(unidecode).map(mapper)
    )
    return df

In [ ]:
def converter_tipos(df, dtypes=None):
    if dtypes:
        for col, tipo in dtypes.items():
            if col in df.columns and df[col].dtype == object and tipo in [int, float]:
                df[col] = (
                    df[col]
                    .str.replace('.', '', regex=False)
                    .str.replace(',', '.', regex=False)
                    .astype(float)
                    .astype(tipo)
                )
    return df

In [ ]:
def merge_subs_ano(df: pd.DataFrame, df_subs_ano: pd.DataFrame) -> pd.DataFrame:
    
    return df.merge(
        df_subs_ano,
        how='left',
        on=['sub.NOME', 'ano']
    )

In [ ]:
def pipeline_completa(df, subs_qlik, df_subs_ano, sub_col_name='região', dtypes=None):
    # df = padronizar_regiao(df, sub_col_name)
    if sub_col_name in df.columns:
        mapper = criar_mapper_subprefeituras(df, subs_qlik, sub_col_name)
        df = aplicar_mapper(df, mapper, sub_col_name)
    df = converter_tipos(df, dtypes)
    df = merge_subs_ano(df, df_subs_ano)
    return df


# Carregando os dados extraídos no notebook anterior

Neste notebook, vamos utilizar os dados extraídos e salvos pelo notebook `05 ObservaSampa - extração.ipynb`.

Primeiro, vamos carregar os dados de subprefeituras que serão utilizados no Qlik Sense.

In [ ]:
output_dir = path.join('dados', 'urbanismo')
if not path.exists(output_dir):
    makedirs(output_dir)

In [ ]:
df_his = pd.read_csv(path.join(output_dir, 'his_entregue_original.csv'),
                     sep=';', decimal=',', encoding='utf8', dtype=str)

In [ ]:

df_tpu = pd.read_csv(path.join(output_dir, 'tpu_emitido_original.csv'),
                     sep=';', decimal=',', encoding='utf8', dtype=str)

In [ ]:

df_110107 = pd.read_csv(path.join(output_dir, 'indicador_110107.csv'),
                        sep=';', decimal=',', encoding='utf8', dtype=str)

In [ ]:

df_110102 = pd.read_csv(path.join(output_dir, 'indicador_110102.csv'),
                        sep=';', decimal=',', encoding='utf8', dtype=str)

In [ ]:

df_110106 = pd.read_csv(path.join(output_dir, 'indicador_110106.csv'),
                        sep=';', decimal=',', encoding='utf8', dtype=str)

In [ ]:
df_010502 = pd.read_csv(path.join(output_dir, 'indicador_010502.csv'),
                        sep=';', decimal=',', encoding='utf8', dtype=str)

In [ ]:
df_110108 = pd.read_csv(path.join(output_dir, 'indicador_110108.csv'),
                         sep=';', decimal=',', encoding='utf8', dtype=str)

In [ ]:
indicador_domicilios_nao_ocupados_df = pd.read_csv(path.join(output_dir, 'indicador_domicilios_nao_ocupados.csv'),
                        sep=';', decimal=',', encoding='utf8', dtype=str)

In [ ]:
df_his.dtypes
df_tpu.dtypes
df_110107.dtypes
df_110102.dtypes
df_110106.dtypes
df_010502.dtypes
df_110108.dtypes
indicador_domicilios_nao_ocupados_df.dtypes

## CSV de Subprefeituras do Qlik

In [ ]:
url_subs = environ.get('CSV_SUBPREFEITURAS_QLIK')
df_subs = pd.read_csv(url_subs)
df_subs

In [ ]:
df_subs = df_subs[['sub.CODIGO', 'sub.NOME']]
df_subs

## Chave composta subprefeitura-ano

Como 3 tabelas possuem valores para mais de um ano, também vale a pena a criação de uma chave composta entre subprefeitura e ano. A tabela que possui mais períodos é a tabela da meta 12 do Programa de Metas, com os anos de 2021, 2022, 2023 e 2024. Vamos criar uma tabela com o produto cartesiano entre subprefeituras e anos.

In [ ]:
df_subs_ano = (
    df_subs[['sub.NOME']]
    .merge(pd.Series(data=[2021, 2022, 2023, 2024], name='ano'),
           how='cross')
)

df_subs_ano.loc[:, 'subprefeitura-ano'] = (
    df_subs_ano.loc[:, 'sub.NOME'] + ' | ' + df_subs_ano.loc[:, 'ano'].astype(str)
)

df_subs_ano

## Subprefeituras Qlik

In [ ]:
subs_qlik = df_subs['sub.NOME'].unique().tolist()
subs_qlik.sort()
subs_qlik

## Produção de habitação de interesse social

In [ ]:
df_his = pipeline_completa(
    df=df_his,
    subs_qlik=subs_qlik,
    df_subs_ano=df_subs_ano,
    dtypes={'qtd_unidades': int, 'ano': int}
)

In [ ]:
df_his

## Número de termos de Permissão de Uso (TPU) emitidos em nome da mulher da familia

In [ ]:
df_tpu

In [ ]:
df_tpu['ano'] = df_tpu['ano'].astype(int)
df_tpu['qtd_termos'] = (
    df_tpu['qtd_termos']
    .str.replace(',', '.')
    .astype(float)
    .astype(int)
)
df_tpu

## Indicador 11.01.08

In [ ]:
df_110108 = pipeline_completa(
    df_110108,
    subs_qlik=subs_qlik,
    df_subs_ano=df_subs_ano,
    dtypes={'qtd_familias': int, 'ano': int}
)

## Indicador 11.01.07

In [ ]:
df_110107 = pipeline_completa(
    df_110107,
    subs_qlik=subs_qlik,
    df_subs_ano=df_subs_ano,
    dtypes={'qtd_domicilios': int, 'ano': int}
    )

In [ ]:
df_110107

## Indicador 11.01.06

In [ ]:
df_110106 = pipeline_completa(
    df=df_110106,
    subs_qlik=subs_qlik,
    df_subs_ano=df_subs_ano,
    dtypes={'qtd_familias': int, 'ano': int}
)

In [ ]:
df_110106

## Indicador 11.01.02

In [ ]:
df_110102 = pipeline_completa(
    df=df_110102,
    subs_qlik=subs_qlik,
    df_subs_ano=df_subs_ano,
    dtypes={'qtd_familias': int, 'ano': int}
)

In [ ]:
df_110102

## Indicador 01.05.02

In [ ]:
df_010502 = pipeline_completa(
    df=df_010502,
    subs_qlik=subs_qlik,
    df_subs_ano=df_subs_ano,
    dtypes={'qtd_familias': int, 'ano': int}
)

In [ ]:
df_010502

## Indicador domicilios não ocupados (censo 2022)

In [ ]:
df = indicador_domicilios_nao_ocupados_df.copy()

In [ ]:
df["cd_subprefeitura"] = df["cd_subprefeitura"].astype(str).str.zfill(2)
df["nm_subprefeitura"] = df["nm_subprefeitura"].astype(str).str.strip()

In [ ]:
num_cols = [
    "domicilios_particulares_nao_ocupados",
    "total_domicilios_particulares",
    "domicilios_particulares_ocupados",
]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df["tx_nao_ocupados_pct"] = (
    np.where(df["total_domicilios_particulares"].gt(0),
             df["domicilios_particulares_nao_ocupados"] / df["total_domicilios_particulares"] * 100,
             np.nan)
)

In [ ]:
cols_final = [
    "cd_subprefeitura",
    "nm_subprefeitura",
    "domicilios_particulares_nao_ocupados",
    "domicilios_particulares_ocupados",
    "total_domicilios_particulares",
    "tx_nao_ocupados_pct",
]

In [ ]:
df = df[cols_final].sort_values(["cd_subprefeitura"]).reset_index(drop=True)

In [ ]:
df_long = df[["cd_subprefeitura", "nm_subprefeitura", "domicilios_particulares_nao_ocupados"]].rename(
    columns={"domicilios_particulares_nao_ocupados": "valor"}
)

In [ ]:
df_long["id_observa"] = 999 
df_long = df_long[["id_observa", "cd_subprefeitura", "nm_subprefeitura", "valor"]]

In [ ]:
df_domicilios_nao_ocupados_tratado = df
df_domicilios_nao_ocupados_long = df_long

In [ ]:
df

In [ ]:
df_long

# Armazenamento

Finalmente, vamos exportar os dados em formato csv compatível com o Qlik e no padrão do excel para português do Brasil.

In [ ]:
base_path = path.join('data_output', 'urbanismo')

if not path.exists(base_path):
    makedirs(base_path)

for name, df in [('producao-his', df_his),
                 ('emissoes-tpu', df_tpu),
                 ('subprefeitura-ano', df_subs_ano),
                 ('indicador_110107', df_110107),
                 ('indicador_110102', df_110102),
                 ('indicador_110106', df_110106),
                 ('indicador_010502', df_010502),
                 ('indicador_110108', df_110108),
                 ('indicador_domicilios_nao_ocupados', df),
                 ('indicador_domicilios_nao_ocupados_long', df_long),
                 ]:
    filepath = path.join(base_path, f'{name}.csv')

    df.to_csv(filepath,
              index=False,
              sep=';',
              decimal=',',
              encoding='latin1')